It is a python library for data validation

In [ ]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel): # represent the ideal schema for data
    # name: str = Field(..., min_length=2, max_length=50)  # name must be between 2 and 50 characters
    name: Annotated[str, Field(min_length=2, max_length=50, title='Name of the patient',
     description='Name must be between 2 and 50 characters', examples=['John Doe', 'Jane Smith'])]  # alternative syntax using Annotated

    email: EmailStr
    linkedin_url: AnyUrl
    age: int = Field(..., gt=0, lt=150, description="Age must be between 1 and 149")  # age must be positive and less than 150
    # weight: float = Field(..., gt=0, description="Weight must be a positive number")
    weight: Annotated[float, Field(gt=0, strict=True, description="Weight must be a positive number")]  # alternative syntax using Annotated
    # strict = True ensures that only float values are accepted, not floats in string format

    # married: bool # bool = False  # default value can also be set
    married: Annotated[bool, Field(default=False, description="Marital status of the patient")]  # alternative syntax using Annotated
    allergies: Optional[List[str]] = Field(default_factory=list, max_items=5)  # default empty list
    contact_details: Optional[Dict[str, str]] = Field(default_factory=dict)  # default empty dict

    

    # default values required for optional 
    # custom data validation can also be added here

patient_info = {'name': 'nitish','email': 'nitish@example.com','linkedin_url':'https://www.linkedin.com/in/nitish', 'age': 30, 'weight': 44.5, 'married': True,
                'allergies': ['pollen', 'dust'], 'contact_details': {'phone': '123-456-7890'}}

patient1 = Patient(**patient_info) # object created for use

def insert_patient_data(xyz: Patient):
    print(xyz.name)
    print(xyz.age)
    print(xyz.married)
    print(xyz.weight)
    print('Inserted')

insert_patient_data(patient1)




nitish
30
True
44.5
Inserted


In [21]:
# field validators 
from pydantic import field_validator, BaseModel, EmailStr

class PatientWithValidation(BaseModel):
    name: str
    age: int
    weight: float
    email: EmailStr

    @field_validator('age') # if mode = 'before' is not specified, it defaults to 'after'
    @classmethod
    def age_must_be_adult(cls, value):
        if value < 18:
            raise ValueError('Age must be at least 18')
        return value
    
    @field_validator('email')
    @classmethod
    def email_must_be_company_domain(cls, value):
        valid_domains = ['hdfc.com', 'icici.com']
        domain_name = value.split('@')[-1]
        if domain_name not in valid_domains:
            raise ValueError(f'Email domain must be one of the following: {valid_domains}')
        return value

    @field_validator('name')
    @classmethod
    
    def name_must_be_alphabetic(cls, value):
        value = value.upper()
        if not value.replace(' ', '').isalpha():
            raise ValueError('Name must contain only alphabetic characters and spaces')
        return value
    
    @field_validator('weight')
    @classmethod
    def weight_must_be_realistic(cls, value):
        if value <= 0 or value > 500:
            raise ValueError('Weight must be a positive number less than or equal to 500')
        return value
    

patient_info = {'name': 'nitish','email': 'nitish@hdfc.com', 'age': 30, 'weight': 44.5}

patient1 = PatientWithValidation(**patient_info) # object created for use

def insert_patient_data(xyz: PatientWithValidation):
    print(xyz.name)
    print(xyz.age)
    print(xyz.weight)
    print(xyz.email)
    print('Inserted')

insert_patient_data(patient1)

NITISH
30
44.5
nitish@hdfc.com
Inserted


In [22]:
# Model validators
from pydantic import model_validator, BaseModel, EmailStr

class PatientWithValidation(BaseModel):
    name: str
    age: int
    weight: float
    email: EmailStr
    contact_number: int

    @model_validator(mode='before')
    @classmethod
    def check_age_if_old(cls, values):
        age = values.get('age')
        contact_number = values.get('contact_number')
        if age and age > 60:
                if contact_number is None or len(str(contact_number)) != 10:
                     raise ValueError('Contact number must be provided and must be 10 digits long for patients over 60 years old')
        return values
    
patient_info = {'name': 'nitish','email': 'nitish@hdfc.com', 'age': 70, 'weight': 44.5, 'contact_number': 1234567890}

patient1 = PatientWithValidation(**patient_info) # object created for use
def insert_patient_data(xyz: PatientWithValidation):
    print(xyz.name)
    print(xyz.age)
    print(xyz.weight)
    print(xyz.email)
    print(xyz.contact_number)
    print('Inserted')
    
insert_patient_data(patient1)

nitish
70
44.5
nitish@hdfc.com
1234567890
Inserted


In [29]:
# Computed fields
from pydantic import computed_field, BaseModel

class PatientWithComputedField(BaseModel):
    name: str
    age: int
    weight_kg: float
    height_cm: float

    @computed_field
    @property
    def weight_lbs(self) -> float:
        return self.weight_kg * 2.20462
    @computed_field
    @property
    def bmi(self) -> float:
        height_m = self.height_cm / 100
        return self.weight_kg / (height_m ** 2)
    
patient_info = {'name': 'nitish','age': 30, 'weight_kg': 70.0, 'height_cm': 175.0}
patient1 = PatientWithComputedField(**patient_info) # object created for use
def insert_patient_data(xyz: PatientWithComputedField):
    print(xyz.name)
    print(xyz.age)
    print(xyz.weight_kg)
    print(xyz.height_cm)
    print(xyz.weight_lbs)
    print(xyz.bmi)
    print('Inserted')


insert_patient_data(patient1)

nitish
30
70.0
175.0
154.3234
22.857142857142858
Inserted


In [ ]:
# nested models
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    gender: str
    address: 'Address'  # forward reference
    
class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

patient_info = {'name': 'nitish','age': 30, 'gender': 'Male',
                'address': {'street': '123 Main St', 'city': 'Mumbai', 'state': 'MH', 'zip_code': '400001'}}
patient1 = Patient(**patient_info) # object created for use

# serialization
temp = patient1.model_dump(include={'name', 'age', 'gender', 'address'}) # can also be in json using model_dump_json()
print(temp)

temp_address = patient1.model_dump(exclude={'name', 'age', 'gender'})
print(temp_address)

# exclude_unset for only those fields which were set during object creation (default values will be excluded)
# temp_unset = patient1.model_dump(exclude_unset=True)

{'name': 'nitish', 'age': 30, 'gender': 'Male', 'address': {'street': '123 Main St', 'city': 'Mumbai', 'state': 'MH', 'zip_code': '400001'}}
{'address': {'street': '123 Main St', 'city': 'Mumbai', 'state': 'MH', 'zip_code': '400001'}}
